In [ ]:
## code authors: katharina lichter, philip kollmannsberger, university and university hospital of wuerzburg, 2022.

## this code is used in the manuscript "Ultrastructural analysis of wildtype and RIM1α knock-out active zones in a large cortical synapse"
## by k lichter, mm paul, m pauli, s schoch, p kollmannsberger, c stigloher, m heckmann, a-l sirén, 2022.

## notebook for calculation of synaptic vesicle (SV) distances to the active zone (AZ) center of mass (com) and AZ edges.

# please import relevant python packages via anaconda navigator or anaconda prompt.

import numpy as np
import math as math
import scipy as sp
from scipy.spatial import distance
from shapely.geometry import LineString, MultiPoint, Point
from shapely.ops import transform, split

##############################################################################

## please import active zone information which is extracted from individual IMOD models (https://bio3d.colorado.edu/imod/). 
# please note that coordinate data are provided in text files. in case of SV data, the SV radius represents a separate column.

# step 1: load the coordinates.
pts = np.loadtxt(fname = 'az-test_docked-sv-pool.txt', usecols = [1,2,3,4,5,6,7]);
linepts = np.loadtxt(fname = 'az-test_area.txt', usecols = [1,2,3,4,5,6]);

# step 2: add the pixel size of the tomogram. this information can be found in your original tilt series.
px_factor = 0.287 

## step 3: distance calculations to AZ com and AZ edge(s).
# calculation of mean coordinate for x, y and z.
AZ_com_x = np.mean(linepts[:,3])
AZ_com_y = np.mean(linepts[:,4])
AZ_com_z = np.mean(linepts[:,5])

# definition of AZ com as 3D point.
AZ_com = Point(AZ_com_x, AZ_com_y, AZ_com_z);

# loop for distance calculation.
for z in np.unique(pts[:,5]):
    
    z_value = np.array(z)
    linepts_z = linepts[linepts[:,5]==z,:];
    
    if linepts_z.shape[0]>0:
        
        line = LineString(linepts_z[:,3:5])
        pts_z = pts[pts[:,5]==z,:]
        vesicle = MultiPoint(pts_z[:,3:5])

# projection of 3D vesicle centers onto the AZ membrane. 

        for p in list(vesicle):
            
            proj_p = Point(line.interpolate(line.project(p)))
            print(proj_p)
            np_proj_p = np.array(proj_p)
            proj_p_3D = transform(lambda x, y: (x, y, z_value), proj_p)
            
            np_line_coords = np.array(line.coords)
            np_line_coords_y = np_line_coords[:,1]
            
# integration of projected 3D vesicle centers into the AZ line profile.
            
            index_in_line = (min(range(len(np_line_coords_y)), key=lambda i: abs(np_line_coords_y[i]-np_proj_p[1])))+1
            np_new_line = np.insert(np_line_coords, index_in_line, np_proj_p, axis=0)
            new_line = LineString(np_new_line)
            print(new_line)

## distance calculation of projected 3D vesicle centers alongside the individual AZ membrane profile to both AZ edges (2D).
            point_to_edge = split(new_line, proj_p)
            line_split_1 = point_to_edge[0]
            line_split_2 = point_to_edge[1]
# calculation of absolute distances.
            az_edge_1 = (line_split_1.length) * px_factor
            print(az_edge_1)
            az_edge_2 = (line_split_2.length) * px_factor
            print(az_edge_2)
# calculation of relative distances (normalized to individual AZ profile length).           
            az_edge_factor_1 = line_split_1.length / new_line.length
            print(az_edge_factor_1)
            az_edge_factor_2 = line_split_2.length / new_line.length
            print(az_edge_factor_2)
# 3D distance of projected 3D vesicle centers to the AZ com.         
            distance_com = (math.sqrt((proj_p_3D.x - AZ_com.x)**2 + (proj_p_3D.y - AZ_com.y)**2 + (proj_p_3D.z - AZ_com.z)**2)) * px_factor
            print(distance_com)   